In [ ]:
from google.colab import files
files.upload()

Saving aligner.py to aligner.py


{'aligner.py': b'import numpy as np\r\nimport PIL\r\nimport PIL.Image\r\nimport scipy\r\nimport scipy.ndimage\r\nimport dlib\r\nfrom pathlib import Path\r\n\r\n\r\n"""\r\nbrief: face alignment with FFHQ method (https://github.com/NVlabs/ffhq-dataset)\r\nauthor: lzhbrian (https://lzhbrian.me)\r\ndate: 2020.1.5\r\nnote: code is heavily borrowed from\r\n    https://github.com/NVlabs/ffhq-dataset\r\n    http://dlib.net/face_landmark_detection.py.html\r\n\r\nrequirements:\r\n    apt install cmake\r\n    conda install Pillow numpy scipy\r\n    pip install dlib\r\n    # download face landmark model from:\r\n    # http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2\r\n"""\r\n\r\n\r\ndef get_landmark(filepath, predictor):\r\n    """get landmark with dlib\r\n    :return: np.array shape=(68, 2)\r\n    """\r\n    detector = dlib.get_frontal_face_detector()\r\n\r\n    img = dlib.load_rgb_image(filepath)\r\n    dets = detector(img, 1)\r\n    filepath = Path(filepath)\r\n    print(f"{file

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
import imageio
import requests
import bz2
from PIL import Image
import torch
import torchvision.transforms as transforms
import dlib
#from pix2pixHD.data.base_dataset
from base_dataset import __scale_width
#from pix2pixHD.models.networks 
from networks import define_G
#import pix2pixHD.util.util
import util as util
from aligner import align_face

import matplotlib.pyplot as plt
%matplotlib inline



In [5]:
def unpack_bz2(src_path):
    data = bz2.BZ2File(src_path).read()
    dst_path = src_path[:-4]
    with open(dst_path, 'wb') as fp:
        fp.write(data)
    return dst_path

def download(url, file_name):
    with open(file_name, "wb") as file:
        response = requests.get(url)
        file.write(response.content)

In [6]:
shape_model_url = 'http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2'
shape_model_path = 'landmarks.dat'
download(shape_model_url, shape_model_path)
shape_predictor = dlib.shape_predictor(unpack_bz2(shape_model_path))

def get_eval_transform(loadSize=512):
    transform_list = []
    transform_list.append(transforms.Lambda(lambda img: __scale_width(img,
                                                                      loadSize,
                                                                      Image.BICUBIC)))
    transform_list += [transforms.ToTensor()]
    transform_list += [transforms.Normalize((0.5, 0.5, 0.5),
                                            (0.5, 0.5, 0.5))]
    return transforms.Compose(transform_list)

transform = get_eval_transform()

config_G = {
    'input_nc': 3,
    'output_nc': 3,
    'ngf': 64,
    'netG': 'global',
    'n_downsample_global': 4,
    'n_blocks_global': 9,
    'n_local_enhancers': 1,
    'norm': 'instance',
}

weights_path_to_female = '/content/drive/MyDrive/to_female_net_G.pth'
weights_path_to_male ='/content/drive/MyDrive/to_male_net_G.pth'

In [7]:
def gender_swap():
  img_filename="/content/static/uploaded_photo/image.jpg"
  aligned_img = align_face(img_filename, shape_predictor)[0]
  img = transform(aligned_img).unsqueeze(0)

  model_to_male = define_G(**config_G)
  pretrained_dict_to_male = torch.load(weights_path_to_male)

  model_to_male.load_state_dict(pretrained_dict_to_male)
  model_to_male.cuda();

  model_to_female = define_G(**config_G)
  pretrained_dict_to_female = torch.load(weights_path_to_female)

  model_to_female.load_state_dict(pretrained_dict_to_female)
  model_to_female.cuda();
 
  with torch.no_grad():
    out_male = model_to_male(img.cuda())
    out_female = model_to_female(img.cuda())

  out_male = util.tensor2im(out_male.data[0])
  out_female = util.tensor2im(out_female.data[0])

  imageio.imsave(RESULTS_FOLDER+'/result_male.jpg', out_male)
  imageio.imsave(RESULTS_FOLDER+'/result_female.jpg', out_female)

In [9]:
# !pip install flask_ngrok
# !pip install flask_restplus

     |████████████████████████████████| 2.5MB 8.7MB/s 
     |████████████████████████████████| 61kB 9.7MB/s 


In [10]:
from flask import Flask
from flask_ngrok import run_with_ngrok
import os
if not os.path.exists('/content/static/uploaded_photo'):
    os.makedirs('/content/static/uploaded_photo')
UPLOAD_FOLDER = '/content/static/uploaded_photo'
if not os.path.exists('/content/static/results'):
    os.makedirs('/content/static/results')
RESULTS_FOLDER = '/content/static/results'
app = Flask(__name__)
run_with_ngrok(app)
app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024

ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg'])

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

In [13]:
import werkzeug
werkzeug.cached_property = werkzeug.utils.cached_property
from flask_restplus import Api, Resource
from werkzeug.datastructures import FileStorage

import os

api = Api(app)
upload_parser = api.parser()
upload_parser.add_argument('file',
                           location='files',
                           type=FileStorage)

@api.route('/upload/')
@api.expect(upload_parser)
class File(Resource):
    def post(self):
        args = upload_parser.parse_args()
        file = args.get('file')
        assert allowed_file(file.filename)
        path = os.path.join(UPLOAD_FOLDER, "image.jpg")
        file.save(path)
        gender_swap()
        return "result was saved at {}".format(UPLOAD_FOLDER)

AssertionError: ignored

In [15]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://3d29479b1de0.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [22/Apr/2021 01:41:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 01:41:15] "GET /swaggerui/droid-sans.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 01:41:15] "GET /swaggerui/swagger-ui.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 01:41:15] "GET /swaggerui/swagger-ui-standalone-preset.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 01:41:15] "GET /swaggerui/swagger-ui-bundle.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 01:41:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 01:41:18] "GET /swaggerui/swagger-ui-bundle.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 01:41:18] "GET /swaggerui/swagger-ui-standalone-preset.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 01:41:18] "GET /swaggerui/swagger-ui.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 01:41:18] "GET /swaggerui/droid-sans.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 01:41:20] "GET /swagger.json HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 01:41:20] "GET /swaggerui/favicon-32x32.png HTTP/1.1" 200 -


image.jpg: Number of faces detected: 1
GlobalGenerator(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
    (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (3): ReLU(inplace=True)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (11): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 1024, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (14): InstanceNor

127.0.0.1 - - [22/Apr/2021 01:41:46] "POST /upload/ HTTP/1.1" 200 -
